In [ ]:
import pandas as pd

# Load Tesla news dataset - using read_excel for .xlsx files
news_df = pd.read_excel(
    "/kaggle/input/tesla-news-2010-2020/tesla_news_articles_2010_2020.xlsx",
    dtype={
        "title": str,
        "snippet": str,
        "lead_paragraph": str,
        "keywords": str,
        "section": str,
        "source": str,
        "url": str
    }
).fillna('')  # Replace NaN with empty strings

# Convert date column to datetime format
news_df["date"] = pd.to_datetime(news_df["date"])

# Sort by date
news_df = news_df.sort_values("date")

# Group by date (concatenating text fields)
news_df = news_df.groupby("date").agg({
    "title": lambda x: " | ".join(x),
    "snippet": lambda x: " | ".join(x),
    "lead_paragraph": lambda x: " | ".join(x),
    "keywords": lambda x: ", ".join(x),
    "section": lambda x: ", ".join(x),
    "source": lambda x: ", ".join(x),
    "url": lambda x: ", ".join(x)
}).reset_index()

# Forward-fill missing dates
stock_dates = pd.date_range(start=news_df["date"].min(), end=news_df["date"].max(), freq="D")
news_df = news_df.set_index("date").reindex(stock_dates).fillna(method="ffill").reset_index()
news_df.rename(columns={"index": "date"}, inplace=True)

# Save the processed dataset
news_df.to_csv("tesla_news_processed.csv", index=False)

print(f"Preprocessed dataset saved with {len(news_df)} rows")

In [ ]:
# Load stock dataset to get valid trading dates
stock_df = pd.read_csv("/kaggle/input/tesla-stock-data-from-2010-to-2020/TSLA.csv")  
stock_df["Date"] = pd.to_datetime(stock_df["Date"])
valid_trading_dates = set(stock_df["Date"])

# Filter news to keep only trading days
news_df = news_df[news_df["date"].isin(valid_trading_dates)]

# Save the corrected dataset
news_df.to_csv("tesla_news_filtered.csv", index=False)

print(f"Filtered dataset saved with {len(news_df)} rows (matching stock trading days)")


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

# Load preprocessed Tesla news dataset
news_df = pd.read_csv("tesla_news_filtered.csv")
news_df["date"] = pd.to_datetime(news_df["date"])

# Concatenate text fields
news_df["full_text"] = news_df["title"] + " " + news_df["snippet"] + " " + news_df["lead_paragraph"]

# Load pre-trained sentence-transformers model (e.g., "all-MiniLM-L6-v2")
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Compute embeddings
news_df["embedding"] = news_df["full_text"].apply(lambda text: embed_model.encode(text, convert_to_numpy=True))

# Convert list of arrays to a 2D NumPy array
embedding_matrix = np.vstack(news_df["embedding"].values)

# Save embeddings to CSV
news_embeddings_df = pd.DataFrame(embedding_matrix)
news_embeddings_df.insert(0, "date", news_df["date"])  # Insert date column
news_embeddings_df.to_csv("news_embeddings.csv", index=False)

print(f"News embeddings saved with shape: {embedding_matrix.shape}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

# Load news embeddings dataset
news_embeddings_df = pd.read_csv("news_embeddings.csv")
news_embeddings_df["date"] = pd.to_datetime(news_embeddings_df["date"])

# Extract only the embedding columns (assume embeddings start from column index 1)
embedding_matrix = news_embeddings_df.iloc[:, 1:].values  # Exclude date column

# Apply PCA to reduce embeddings from 384 to 50 dimensions
pca = PCA(n_components=5)
reduced_embeddings = pca.fit_transform(embedding_matrix)

# Convert back to DataFrame
reduced_embeddings_df = pd.DataFrame(reduced_embeddings, columns=[f"pca_{i+1}" for i in range(5)])
reduced_embeddings_df.insert(0, "date", news_embeddings_df["date"])  # Reinsert date column

# Save reduced embeddings
reduced_embeddings_df.to_csv("news_embeddings_pca.csv", index=False)

print(f"Reduced news embeddings saved with shape: {reduced_embeddings_df.shape}")
import pandas as pd

# Load stock dataset
stock_df = pd.read_csv("/kaggle/input/tesla-stock-data-from-2010-to-2020/TSLA.csv")
stock_df["Date"] = pd.to_datetime(stock_df["Date"])

# Load reduced news embeddings dataset
news_embeddings_df = pd.read_csv("news_embeddings_pca.csv")
news_embeddings_df["date"] = pd.to_datetime(news_embeddings_df["date"])

# Merge stock prices with reduced news embeddings on date
merged_df = pd.merge(stock_df, news_embeddings_df, left_on="Date", right_on="date", how="left").drop(columns=["date"])

# Save the final dataset
merged_df.to_csv("tesla_stock_news_fused_pca.csv", index=False)

print(f"Merged dataset saved with {len(merged_df)} rows and {merged_df.shape[1]} columns")


In [ ]:
import pandas as pd

# Load stock dataset
stock_df = pd.read_csv("/kaggle/input/tesla-stock-data-from-2010-to-2020/TSLA.csv")
stock_df["Date"] = pd.to_datetime(stock_df["Date"])

# Load reduced news embeddings dataset
news_embeddings_df = pd.read_csv("news_embeddings_pca.csv")
news_embeddings_df["date"] = pd.to_datetime(news_embeddings_df["date"])

# Merge stock prices with reduced news embeddings on date
merged_df = pd.merge(stock_df, news_embeddings_df, left_on="Date", right_on="date", how="left").drop(columns=["date"])

# Save the final dataset
merged_df.to_csv("tesla_stock_news_fused_pca.csv", index=False)

print(f"Merged dataset saved with {len(merged_df)} rows and {merged_df.shape[1]} columns")


In [ ]:
df = pd.read_csv('/kaggle/working/tesla_stock_news_fused_pca.csv')
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date").set_index("Date")
# Add critical columns
df["time_idx"] = np.arange(len(df))  # Sequential integer index
df["group_id"] = "TSLA"  # Single time series identifier

# Create ALL required time features (FIX for KeyError)
df["Day_of_week"] = df.index.dayofweek  # Monday=0, Sunday=6
df["Month"] = df.index.month  # 1-12
df["Year"] = df.index.year  # 2010-2020
df["Day_of_month"] = df.index.day  # 1-31
df["Week_of_year"] = df.index.isocalendar().week  # 1-52

# Target: log returns (better than raw prices)
df["Target"] = np.log(df["Close"]).diff()  

In [ ]:
print(df.columns.tolist())

In [ ]:
# Keep only essential features
keep_cols = ["Target", "Close", "Volume", "time_idx", "group_id", 
             "Day_of_week", "Month", "Year", "Day_of_month", "Week_of_year","pca_1","pca_2",
             "pca_3","pca_4","pca_5"]
df = df[keep_cols].dropna()

In [ ]:
# %% [code]
# Train/validation/test split
test_cutoff = df.index[-365]  # Last year for testing
val_cutoff = test_cutoff - pd.Timedelta(days=365)  # Year before test

train_df = df.loc[df.index < val_cutoff]
val_df = df.loc[(df.index >= val_cutoff) & (df.index < test_cutoff)]
test_df = df.loc[df.index >= test_cutoff]

# %% [code]
# Create dataset
max_encoder_length = 60  # Lookback window
max_prediction_length = 7  # Forecast horizon

training = TimeSeriesDataSet(
    train_df,
    time_idx="time_idx",
    target="Target",
    group_ids=["group_id"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    time_varying_known_reals=["Day_of_week", "Month", "Year", "Day_of_month", "Week_of_year"],
    time_varying_unknown_reals=["Target", "Close", "Volume","pca_1", "pca_2", "pca_3", "pca_4", "pca_5"],
    static_categoricals=[],
    static_reals=[],
    target_normalizer=None,
    add_relative_time_idx=True,
)

validation = TimeSeriesDataSet.from_dataset(training, val_df, predict=True)
# %% [code]
# Create dataloaders
batch_size = 64
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

In [ ]:
# %% [code]
# Configure TFT model
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.001,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.2,
    hidden_continuous_size=8,
    loss=QuantileLoss(),
    optimizer="adamw",
)

# %% [code]
# Train with early stopping
trainer = pl.Trainer(
    max_epochs=50,
    accelerator="auto",
    enable_model_summary=True,
    gradient_clip_val=0.1,
    callbacks=[
        pl.callbacks.EarlyStopping(monitor="val_loss", patience=5),
        pl.callbacks.LearningRateMonitor()
    ],
)

trainer.fit(tft, train_dataloader, val_dataloader)

# %% [code]
# Evaluate on test set - CORRECTED VERSION
test_dataset = TimeSeriesDataSet.from_dataset(training, test_df, predict=True)
test_dataloader = test_dataset.to_dataloader(train=False, batch_size=batch_size, num_workers=3)

In [ ]:
# %% [code]
# Corrected Evaluation and Prediction Conversion

# Get predictions - using the right mode
predictions = tft.predict(test_dataloader, mode="raw")
point_predictions = tft.to_prediction(predictions)  # Gets median prediction (quantile=0.5)

# Convert to numpy array and flatten
predicted_returns = point_predictions.cpu().numpy().flatten()

# Calculate available test points
test_points = len(predicted_returns)

# Verify shapes
print(f"Predicted returns shape: {predicted_returns.shape}")
print(f"Available test points: {test_points}")

# Initialize price conversion
predicted_prices = np.zeros(test_points)
current_price = test_df["Close"].iloc[max_encoder_length-1]  # Last known price

# Walk forward conversion
for i in range(test_points):
    predicted_prices[i] = current_price * np.exp(predicted_returns[i])
    current_price = predicted_prices[i]  # Update for next step

# Get corresponding actual prices
actual_prices = test_df["Close"].iloc[max_encoder_length:max_encoder_length+test_points].values

# %% [code]
# Calculate metrics
mae = np.mean(np.abs(predicted_prices - actual_prices))
rmse = np.sqrt(np.mean((predicted_prices - actual_prices)**2))
mape = np.mean(np.abs((actual_prices - predicted_prices)/actual_prices)) * 100

print(f"\nEvaluation Metrics:")
print(f"MAE: ${mae:.2f}")
print(f"RMSE: ${rmse:.2f}")
print(f"MAPE: {mape:.2f}%")
print(f"Test Period: {test_df.index[max_encoder_length].date()} to {test_df.index[-1].date()}")
print(f"Test Points: {test_points} days")

# %% [code]
# Enhanced Plotting
plt.figure(figsize=(14, 7))
plt.plot(test_df.index[max_encoder_length:max_encoder_length+test_points], 
         actual_prices, 'b-', label="Actual Price")
plt.plot(test_df.index[max_encoder_length:max_encoder_length+test_points], 
         predicted_prices, 'r--', label="Predicted Price")
plt.fill_between(test_df.index[max_encoder_length:max_encoder_length+test_points],
                actual_prices * 0.98, actual_prices * 1.02,
                color='gray', alpha=0.2, label="±2% Range")
plt.title(f"Tesla Stock Price Prediction\n{test_points}-Day Test Period", fontsize=14)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Price ($)", fontsize=12)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()